<a href="https://colab.research.google.com/github/SirTee12/LLM-Kaggle-Google-GenAI/blob/main/Prompt_engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install the SDK

In [1]:
!pip uninstall -qqy jupyterlab # uninstall jupyterlab in the case it conflict google colab base image
!pip install -U -q "google-genai==1.7.0" # install google genai library

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 4.9 MB/s eta 0:00:00


import the SDK and some helpers for rendering the output. The `types` in `import types` may include custom data structures, classes or type hints psecifically to work with google AI. These custom types might represent things like model parameters, input/output formats for AI models or config settings.

In [2]:
from google import genai
from google.genai import types
from google.colab import userdata

import enum
from IPython.display import HTML, Markdown, display
import typing_extensions as typing

## API Call Retry Implementation
Set up a retry helper. This allows you to "Run all" without worrying about per-minute quota


1.  **Imports `retry`:** It imports the `retry` module from `google.api_core`, which provides tools for automatic retries of API calls.

2.  **Defines `is_retriable`:**
    * It creates a lambda function called `is_retriable`.
    * This function checks if an exception (`e`) is an instance of `genai.errors.APIErrpr` and if its error code (`e.code`) is either 429 (Too Many Requests) or 503 (Service Unavailable).
    * It returns `True` if both conditions are met (meaning the API call is considered retriable), and `False` otherwise.

3.  **Applies Retry Logic:**
    * It modifies the `genai.models.Models.generate_content` function by wrapping it with the `retry.retry` function.
    * The `retry.retry` function is configured with the `is_retriable` function as the `predicate`. This tells `retry.retry` to only retry the API call if the `is_retriable` function returns `True` for the encountered exception.
    * Essentially, this replaces the original `generate_content` function with a new version that automatically retries on 429 and 503 errors, making the code more robust to temporary API issues.

In [3]:
from google.api_core import retry

is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in (429, 503))

genai.models.Models.generate_content = retry.Retry(predicate = is_retriable)(genai.models.Models.generate_content)

In [16]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("secret-label-prompt")

ModuleNotFoundError: No module named 'kaggle_secrets'

## Get the API keys and create a client

Retrieves the API key securely and create a client object that will be used to communicate with the generative AI service, using the retrieved API key for authentication.

In [4]:
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY') # retrive the APi key
client = genai.Client(api_key=GOOGLE_API_KEY)   # create a client to interact with the genai application

## Running the First Prompt

In [5]:
# generate a response
response = client.models.generate_content(
    model = 'gemini-2.0-flash',
    contents = "Explain AI to me like I'm a kid"
)

print(response.text)
Markdown(response.text)

Okay, imagine you have a really, really smart puppy! That puppy can learn tricks, right?  You show it something over and over, and eventually, it learns to do it on its own.

AI is like that, but instead of a puppy, it's a computer program.  We teach the computer program how to do things by showing it lots and lots of examples.

*   **Learning by Example:** Let's say you want to teach the computer to recognize cats. You show it thousands of pictures of cats. After seeing so many cat pictures, the computer learns what cats generally look like.
*   **Making Choices:**  Then, you show the computer a new picture and ask, "Is this a cat?" Because it learned from all those examples, it can usually guess correctly!
*   **Helping Us Out:** AI can do lots of things like recognize faces in pictures, suggest what movies you might like to watch, or even help doctors find diseases.

So basically, AI is a computer program that learns from information and can do smart things like a very well-trained 

Okay, imagine you have a really, really smart puppy! That puppy can learn tricks, right?  You show it something over and over, and eventually, it learns to do it on its own.

AI is like that, but instead of a puppy, it's a computer program.  We teach the computer program how to do things by showing it lots and lots of examples.

*   **Learning by Example:** Let's say you want to teach the computer to recognize cats. You show it thousands of pictures of cats. After seeing so many cat pictures, the computer learns what cats generally look like.
*   **Making Choices:**  Then, you show the computer a new picture and ask, "Is this a cat?" Because it learned from all those examples, it can usually guess correctly!
*   **Helping Us Out:** AI can do lots of things like recognize faces in pictures, suggest what movies you might like to watch, or even help doctors find diseases.

So basically, AI is a computer program that learns from information and can do smart things like a very well-trained puppy (but without the need to be walked!).


## Let's start a chat

In [6]:
chat = client.chats.create(model = "gemini-2.0-flash", history = [])
response = chat.send_message('Hello, my name is Ahmad')
print(response.text)

Hello Ahmad! It's nice to meet you. How can I help you today?



In [7]:
response = chat.send_message("can you tell me something interesting about dinosaurs")
print(response.text)

Okay, here's a fascinating fact about dinosaurs:

**The Spinosaurus was likely the first known swimming dinosaur.**

For a long time, dinosaurs were thought to be strictly land-based creatures. However, discoveries in recent years have strongly suggested that the Spinosaurus, a massive theropod dinosaur (larger than a T-Rex), spent a significant amount of its time in the water. Evidence for this includes:

*   **Its dense bones:** Unlike most theropods which had hollow bones for lightness, Spinosaurus had dense bones, which would have helped it stay submerged.
*   **Its paddle-like feet:** These feet were more suited for paddling than walking on land.
*   **Its sail-like structure:** Scientists now believe that the large sail on its back might have been used for stability in the water or for display.
*   **Isotope analysis of its teeth:** Showed similar oxygen isotope ratios to aquatic animals living in the same area.
*   **Fossils found in aquatic environments:** Many Spinosaurus foss

In [8]:
response = chat.send_message("do you still remember my name")
print(response.text)

Yes, Ahmad! I remember your name.



# Chose a model from the Gemini model family

Retrieves a list of available AI models, searches for a specific model named "gemini-2.0-flash," and if found, displays its details in a formatted JSON-like structure. The search stops once the target model is located.

In [9]:
from pprint import pprint

for model in client.models.list():
  if model.name == "models/gemini-2.0-flash":
    pprint(model.to_json_dict())
    break

{'description': 'Gemini 2.0 Flash',
 'display_name': 'Gemini 2.0 Flash',
 'input_token_limit': 1048576,
 'name': 'models/gemini-2.0-flash',
 'output_token_limit': 8192,
 'supported_actions': ['generateContent', 'countTokens'],
 'tuned_model_info': {},
 'version': '2.0'}


## Parameters Exploration

### Output Length

In [10]:
# set the max number of output token to 200
short_config = types.GenerateContentConfig(max_output_tokens = 200)

# create a response
response_short = client.models.generate_content(
    model = 'gemini-2.0-flash',
    config = short_config,
    contents='write a short essay on the importance of education in modern day society.'
)

print(response_short.text)

## The Cornerstone of Progress: Education in Modern Society

In the swirling currents of modern society, a society defined by rapid technological advancements, complex global challenges, and ever-shifting social landscapes, education stands as the unwavering cornerstone of progress. No longer simply the acquisition of knowledge, it is the essential tool that empowers individuals, strengthens communities, and shapes a future brimming with possibility. Its importance transcends mere career prospects; it is fundamental to individual empowerment, economic growth, and societal well-being.

Firstly, education empowers individuals by equipping them with critical thinking skills. In an age saturated with information and misinformation, the ability to analyze, evaluate, and synthesize knowledge is paramount. Education fosters this intellectual independence, enabling individuals to navigate the complexities of the modern world and make informed decisions. It encourages questioning, exploration, 

### Temperature

How much randomness is included when selecting the next word (token) in language models depends on the "temperature" parameter.  A higher temperature causes the model to take into account a greater number of potential words, producing more inventive and diverse results.  On the other hand, a lower temperature forces the model to adhere to the most likely terms, producing language that is more concentrated and predictable.  The model turns completely deterministic when the temperature is set to 0, always choosing the word that is the most likely.  Temperature does not, however, ensure actual randomness; rather, it is a means of directing the model toward more or less random results.

In [11]:
# create a variable to set the temperature
high_temp_config = types.GenerateContentConfig(temperature = 2.0)

for _ in range(5):
  response_temp = client.models.generate_content(
      model = 'gemini-2.0-flash',
      config = high_temp_config,
      contents = 'Pick a random colour... (respond in a single word)'
  )

  if response_temp.text:
    print(response_temp.text, '_' * 25)

Magenta
 _________________________
Orange
 _________________________
Azure.
 _________________________
Magenta
 _________________________
Turquoise
 _________________________


The previous output shows the randomness but we would experiment with temperature value of 0. we can see from the output that it is more direct and precise and there is no randomness.

In [12]:
# create a variable to set the temperature
high_temp_config = types.GenerateContentConfig(temperature = 0)

for _ in range(5):
  response_temp = client.models.generate_content(
      model = 'gemini-2.0-flash',
      config = high_temp_config,
      contents = 'Pick a random colour... (respond in a single word)'
  )

  if response_temp.text:
    print(response_temp.text, '_' * 25)

Azure
 _________________________
Azure
 _________________________
Azure
 _________________________
Azure
 _________________________
Azure
 _________________________


### Top P

In [13]:
model_config = types.GenerateContentConfig(
    temperature = 1.0, # default config
    top_p = 0.95       # default config
)

story_prompt = "You are a senior network engineer. Explain ospf to a 5 year old"
response_topp = client.models.generate_content(
    model = 'gemini-2.0-flash',
    config = model_config,
    contents=story_prompt
)

print(response_topp.text)

Okay, imagine the internet is like a giant neighborhood filled with lots of houses (computers). Each house needs to know how to get to every other house.  

Now, OSPF is like a super-smart mailman!

*   **Everyone knows everyone:** First, all the houses that want to talk to each other get together and make a map of the entire neighborhood. This map shows all the streets (network connections) and houses (computers).

*   **Finding the best path:** Instead of just picking any road, OSPF figures out the *fastest* and *easiest* way to deliver mail (data). It's like finding the shortest route with the least traffic!

*   **If a road closes:** If a street gets blocked (the connection breaks), OSPF is super quick to update the map. All the houses get a new map, so they know to take a different route.

*   **Talking to neighbors:** Houses don't need to yell across the entire neighborhood. They only need to talk to their neighbors (other computers connected to them). This makes things much fast

## Prompting

### Zero-shot Prompt

In [14]:
# set the prompt parameters
model_config = types.GenerateContentConfig(
  temperature = 0.1,
  top_p = 1,
  max_output_tokens = 5
)

# set the prompt
zero_short_prompt = """Classify movie reviews as POSITIVE, NEUTRAL or NEGATIVE.
Review: "Her" is a disturbing study revealing the direction
humanity is headed if AI is allowed to keep evolving,
unchecked. I wish there were more movies like this masterpiece.
Sentiment: """

response_zero_shot = client.models.generate_content(
    model = 'gemini-2.0-flash',
    config = model_config,
    contents = zero_short_prompt
)

print(response_zero_shot.text) # print the output as text

POSITIVE



#### Enum Mode
The Sentiment enum acts as a schema or a constraint. It tells the language model that the expected output should be one of these three predefined sentiment categories.

The `text/x.enum` MIME type indicates that the response should be one of the values defined in the response_schema.

In [15]:
# create a new enumeration class tha inherit from enum.Enum
class Sentiment(enum.Enum):
  POSITIVE = 'positive'
  NEUTRAL = 'neutral'
  NEGATIVE = 'negative'

response_zero_short_enum = client.models.generate_content(
    model = 'gemini-2.0-flash',
    config = types.GenerateContentConfig(

        # tell the model the ezpected output format and that the rsponse should
        # be one of thos defined in the schema
        response_mime_type = 'text/x.enum',

        # set the Sentiment as schema
        response_schema = Sentiment
    ),

    contents = zero_short_prompt
)
print(response_zero_shot.text)

POSITIVE



### Few Short Prompt

In [16]:
few_shot_prompt = """Parse a customer's pizza order into valid JSON:

EXAMPLE:
I want a small pizza with cheese, tomato sauce, and pepperoni.
JSON Response:
```
{
"size": "small",
"type": "normal",
"ingredients": ["cheese", "tomato sauce", "pepperoni"]
}
```

EXAMPLE:
Can I get a large pizza with tomato sauce, basil and mozzarella
JSON Response:
```
{
"size": "large",
"type": "normal",
"ingredients": ["tomato sauce", "basil", "mozzarella"]
}
```

ORDER:
"""

customer_order = 'Give me a large with cheese and pineapple'

response_one_few_shot = client.models.generate_content(
    model = 'gemini-2.0-flash',
    config = types.GenerateContentConfig(
        temperature = 1,
        top_p = 1,
        max_output_tokens = 250
    ),
    contents = [few_shot_prompt, customer_order]
)

print(response_one_few_shot.text)

```json
{
"size": "large",
"type": "normal",
"ingredients": ["cheese", "pineapple"]
}
```



#### JSON Mode

`class PizzaOrder(typing.TypedDict):`: This defines a new class called PizzaOrder that inherits from typing.TypedDict. TypedDict is used to create dictionary types where the keys and their corresponding value types are known

Purpose of the `TypedDict` is to act as a schema. It defines the structure and data types of the JSON output we expect from the language model. This allows us to work with the model's response in a structured and predictable way.


In [17]:
class PizzaOrder(typing.TypedDict):
  size:str
  ingredients: list[str]
  type:str

response_json = client.models.generate_content(
    model = 'gemini-2.0-flash',
    config = types.GenerateContentConfig(
        temperature = 0.1,
        response_mime_type = 'application/json',
        response_schema = PizzaOrder
    ),

    contents = "Can I have a large dessert pizza with apple and chocolate"
)

print(response_json.text)

{
  "size": "large",
  "ingredients": ["apple", "chocolate"],
  "type": "dessert"
}


## Chain of Thought

Chain of Thought prompting improves the language model's ability to perform complex reasoning tasks. By explicitly requesting the model to show its reasoning process, it's more likely to arrive at the correct answer, as it avoids impulsive or incorrect conclusions.

In [18]:
# set up the prompt
cot_prompt = """When I was 4 years old, my partner was 3 times my age. Now, I
am 20 years old. How old is my partner? let's think step by step."""

response_cot = client.models.generate_content(
    model = 'gemini-2.0-flash',
    contents = cot_prompt)

Markdown(response_cot.text)


Here's how to solve the problem step-by-step:

1. **Find the age difference:** When you were 4, your partner was 3 times your age, meaning they were 4 * 3 = 12 years old.

2. **Calculate the age difference:** The age difference between you and your partner is 12 - 4 = 8 years.

3. **Determine partner's current age:** Since the age difference remains constant, your partner is always 8 years older than you. Now that you are 20, your partner is 20 + 8 = 28 years old.

**Therefore, your partner is currently 28 years old.**
